Notebook elaborado para estudar as principais transformações do campo potencial aplicadas ao método gravimétrico.
Criado para a disciplina Geofísica II da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Esta demonstração utiliza os códigos de transformação de campo potencial implementados no pacote *open source* [fatiando (v0.5)](https://github.com/fatiando/fatiando) (UIEDA; OLIVEIRA Jr; BARBOSA, 2013), distribuído sob os termos da [licença BSD-3](https://github.com/fatiando/fatiando/blob/master/LICENSE.txt). Não foram feitas modificações nos códigos utilizados.

Esta demonstração utiliza a modelagem de dados gravimétricos implementados no pacote *open source* [SimPEG v.14.3](https://github.com/simpeg/simpeg) (COCKETT et al., 2015), distribuído sob os termos da [licença MIT](https://github.com/simpeg/simpeg/blob/master/LICENSE). Não foram feitas modificações nos códigos utilizados.


Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>

<p>&nbsp;</p>
Última modificação: 23/09/2022

---

**Referências:**

UIEDA, L.; OLIVEIRA Jr, V. C.; BARBOSA, V. C. F. Modeling the Earth with Fatiando a Terra. **Proceedings of the 12th Python in Science Conference**, p. 91 - 98, 2013.

COCKETT, R.; KANG, S.; HEAGY, L. J.; PIDLISECKY, A.; OLDENBURG, D. W. SimPEG: An open source framework for simulation and gradient based parameter estimation in geophysical applications. **Computers & Geosciences**, v.85, Part A, p. 142-154, 2015.

# Instruções iniciais

Neste notebook você irá visualizar a anomalia Bouguer gerada por corpos de geometria simples. Para isto leia atentamente as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação
Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.
**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada.

In [ ]:
import modulos.potgrav as pg
import numpy as np

# Passo 02 - Configuração do modelo

Aqui você irá definir primeiramente as dimensões do espaço do modelo, informando o comprimento que você deseja que seu modelo tenha e qual a discretização que deseja para ele. Cada variável na célula abaixo define uma dimensão para um eixo do modelo tridimensional.

Para este exemplo vamos começar com um modelo que tem 2000 m de comprimento (direção X), 2000 m de largura (direção Y) e 260 m de profundidade (direção Z) e é discretizado em elementos de 40.0 x 40.0 x 20.0 m. Configure estes valores executando a célula abaixo.

In [ ]:
eixo_x = [2000.0, 40.0]      # [comprimento, delta]
eixo_y = [2000.0, 40.0]      # [comprimento, delta]
eixo_z = [260.0, 20.0]       # [comprimento, delta]

A modelagem neste exemplo considera os contrastes de densidade para as rochas, e não os seus valores absolutos. A densidade da crosta continental varia entre 2.65 e 2.80 g/cm$^3$, então neste exemplo vamos considerar o valor médio de 2.72 g/cm$^3$ como um valor de referência para os contrastes que serão definidos adiante.

Execute a célula abaixo para gerar o modelo.

In [ ]:
meu_modelo = pg.modelo(eixo_x, eixo_y, eixo_z)

Agora você irá inserir contrastes de densidade no modelo para simular corpos na rocha encaixante sedimentar. Os contrastes serão representados por corpos de geometria simples: um corpo esférico de raio 150.0 m com centro nas coordenadas $(x, y)$ = (-200.0, -200.0) e um corpo cúbico com centro nas coordenadas $(x, y)$ = (-200.0, 200.0). O topo de cada um destes corpos encontra-se à profundidade de -50.0 m. O contraste de densidade do corpo esférico em relação à rocha encaixante é de 0.4 g/cm$^3$, enquanto o contraste de densidade do corpo cúbico é de -0.4 g/cm$^3$. Assim, o corpo esférico tem uma densidade igual a 3.1 g/cm$^3$ e o corpo cúbico tem uma densidade igual a 2.3 g/cm$^3$. Tanto os corpos quanto a rocha encaixante sedimentar encontram-se sobre o embasamento cristalino, que apresenta contraste de densidade em relação à rocha sedimentar de 0.3 g/cm$^3$ (densidade igual a 3.0 g/cm$^3$, portanto).

Execute a célula abaixo para construir o modelo descrito acima.

In [ ]:
# corpo esférico (delta_rho, x_centro, y_centro, z_topo, raio)
meu_modelo.insere_esfera(0.4, -200.0, -200.0, -50.0, 150)

# corpo cúbico (delta_rho, [(xi, xf),(yi, yf),(zi, zf)])
meu_modelo.insere_bloco(-0.4, [(-350, -50), (50, 350), (-200, -50)])

# degrau (delta_rho, [(xi, xf),(yi, yf),(zi, zf)])
meu_modelo.insere_bloco(0.3, [(-1000, 1000), (-1000, 1000), (-260, -200)])
meu_modelo.insere_bloco(0.3, [(200, 1000), (-1000, 1000), (-200, -140)])

Execute a célula abaixo para exibir vistas em corte do modelo e conferir o posicionamento dos corpos em relação ao embasamento cristalino.

In [ ]:
meu_modelo.plota_corte('Y', 200.0)
meu_modelo.plota_corte('Y', -200.0)
meu_modelo.plota_corte('X', -200.0)
meu_modelo.plota_corte('Z', 200.0)

# Passo 03 - Configuração da aquisição

Agora será preciso definir os parâmetros de aquisição de dados, i.e., os pontos onde serão realizadas as medidas de aceleração de gravidade sobre o modelo. Estes dados serão adquiridos em uma grade de pontos equiespaçados distribuídos na superfície do modelo, portanto será preciso definir os pontos inicial e final para o eixo X e para o eixo Y, assim como o número de pontos ao longo de cada direção.

Execute a célula abaixo para definir um levantamento realizado entre as coordenadas -1000.0 e 1000 do eixo X e entre as coordenadas -1000.0 e 1000.0 m do eixo Y, com 40 pontos de aquisição em cada direção. Esta simulação irá considerar também a altura do levantamento em 0.0 m, i.e., o levantamento será feito junto à superfície do modelo.

In [ ]:
x_rx = np.linspace(-1000, 1000.0, 40)
y_rx = np.linspace(-1000.0, 1000.0, 40)
h_rx = 0.0

minha_aquisicao = pg.aquisicao(x_rx, y_rx, h_rx, meu_modelo, componente='gz')
%time minha_aquisicao.fwd(meu_modelo)

### Mapa de anomalia Bouguer

O mapa gerado pela simulação sobre o modelo corresponde ao conjunto de dados resultantes do procedimento de redução dos dados adquiridos em campo, que recebe o nome de mapa de anomalia Bouguer. Este mapa representa a componente z da aceleração da gravidade, uma vez que a influência das densidades das massas topográficas já foi removida (ou teria sido, se o modelo usado no exemplo tivesse topografia).

Execute a célula abaixo para obter o mapa de anomalia Bouguer associado ao modelo configurado.

In [ ]:
minha_aquisicao.plota_mapa([-200.0, -200.0])

# Passo 04 - Transformações do campo

### Continuação para cima

A continuação para cima simula o efeito anômalo causado pelos contrastes de densidade em alturas diferentes daquela em que os dados foram adquiridos. Esta transformação atua sobre o datum onde os dados foram adquiridos deslocando-o para cima, daí o nome desta operação. Isto faz com que a influência das anomalias de curto comprimento de onda seja minimizada, enquanto as anomalias de longo comprimento de onda ainda continuam presentes no dado apresentando uma amplitude ligeiramente menor do que a amplitude original. Trata-se, portanto, de um tipo de filtragem que remove do dado as anomalias de curto comprimento de onda.

Existem duas aplicações principais para este tipo de procedimento: i) colocar diferentes conjuntos de dados em um mesmo datum para que sejam trabalhados em conjunto, como por exemplo quando se tem dados adquiridos em levantamentos aéreo e terrestre sobre uma mesma região, e ii) para estimar a influência do campo regional, que poderá ser utilizado para o cálculo do campo resitual.

No modelo utilizado neste notebook os dados foram adquiridos em terra ($h$ = 0.0 m). Execute a célula abaixo para transportar esta aquisição para uma altura de 500.0 m acima da superfície ($h$ = 500.0 m), obtendo assim um mapa de continuação para cima.

In [ ]:
alt = 500.0        # altura da continuação (em metros)

minha_aquisicao_up = pg.continuacao(minha_aquisicao, alt)
minha_aquisicao_up.plota_mapa([-200.0, -200.0])

### Campo Residual

A análise de um conjunto de dados gravimétricos adquirido com a finalidade de estudar um corpo em particular requer que as anomalias de interesse sejam isoladas para que se possa proceder à sua interpretação. No contexto exploratório, em geral as anomalias de interesse correspondem a anomalias causadas por corpos rasos; entretanto o dado gravimétrico adquirido em campo também está sujeito à influência dos contrastes de densidades mais profundos presentes na crosta terrestre. Para analisar corretamente as anomalias causadas pelos corpos mais rasos faz-se necessário separar as anomalias causadas por fontes mais rasas daquelas causadas por corpos mais profundos.

A separação é feita estimando-se o campo regional, ou seja, a representação do comportamento geral do campo gravimétrico na região do levantamento. Isto pode ser feito de duas formas: i) utilizando-se o método dos quadrados mínimos para ajustar um polinomio bidimensional sobre os dados adquiridos, e ii) através da continuação para cima a uma altitude em que as anomalias de longo comprimento de onda sejam observadas preferencialmente. Uma vez estimado, o campo regional é subtraído dos dados originais, restando no mapa apenas as anomalias de curto comprimento de onda associadas aos corpos mais rasos. Este campo resultante é chamado de campo residual.

Execute a célula abaixo para calcular o campo gravimétrico residual utilizando a continuação para cima executada na etapa anterior. É este mapa que será utilizado nas análises posteriores das anomalias observadas no mapa.

In [ ]:
minha_aquisicao_res = pg.residual(minha_aquisicao, minha_aquisicao_up)
minha_aquisicao_res.plota_mapa([-200.0, -200.0])

**Obs.:** Você irá observar que este mapa não apresenta diferenças significativas em comparação com o mapa de anomalia Bouguer obtido no **Passo 03**. Isto ocorre porque os dados simulados não contém influência de estruturas profundas que seriam eliminadas com a remoção do campo regional e, portanto, não há efetivamente um campo regional a ser removido neste caso em particular. No entanto, optou-se por apresentar esta etapa para ilustrar o procedimento correto, uma vez que a remoção do campo regional é necessária quando se faz a análise de um conjunto de dados reais.

### 1a derivada horizontal

A 1a derivada horizontal auxilia no processo de identificação de falhas subverticais. No exemplo apresentado neste modelo, note que o pico da 1a derivada na direção X coincide com a coordenada do "degrau" na interface entre os sedimentos e o embasamento nesta direção, que é causado pelo deslocamento entre os blocos.

Execute a célula abaixo para calcular a 1a derivada horizontal na direção X dos dados do mapa do campo residual.

In [ ]:
minha_aquisicao_dx = pg.derivada_x(minha_aquisicao_res, ordem=1)
minha_aquisicao_dx.plota_mapa([500.0, -600.0])

### 1a derivada vertical

Enquanto a 1a derivada horizontal permite identificar a localização do plano de falha em uma falha subvertical, a 1a derivada vertical possibilita identificar qual é o bloco em que houve soerguimento e o bloco em que houve rebaixmento. Estes blocos podem ser associados, respectivamente, aos valores positivos e negativos da derivada vertical. Note-se, porém, que esta transformação não permite identificar qual é o tipo de falha (normal ou inversa).

Execute a célula abaixo para calcular a 1a derivada horizontal na direção X dos dados do mapa do campo residual.

In [ ]:
minha_aquisicao_dz1 = pg.derivada_z(minha_aquisicao_res, ordem=1)
minha_aquisicao_dz1.plota_mapa([-200.0, -600.0])

### 2a derivada vertical

Você deve ter observado que a 1a derivada vertical também modificou um pouco o formato da anomalia causada pelos corpos esférico e cúbico. Embora a 1a derivada vertical auxilie neste tipo de análise, a 2a derivada vertical permite identificar os limites destes corpos com maior acurácia. Observe no mapa abaixo como os valores diferentes de zero para cada anomalia ficam muito mais próximos das coordenadas configuradas para as bordas dos corpos no nosso modelo.

Uma outra particularidade da 2a derivada vertical é que ela realça anomalias causadas por fontes rasas, favorecendo-as em relação às anomalias causadas por fontes mais profundas. Isto pode ser observado comparando-se as anomalias causadas pelos dois corpos mencionados anteriormente em relação à anomalia causada pelo deslocamento do bloco mais alto na falha subvertical presente no embasamento.

Execute a célula abaixo para calcular a 2a derivada horizontal na direção X dos dados do mapa do campo residual.

In [ ]:
minha_aquisicao_dz2 = pg.derivada_z(minha_aquisicao_res, ordem=2)
minha_aquisicao_dz2.plota_mapa([-200.0, -200.0])

### Amplitude do Gradiente Total (TGA)

A Amplitude do Gradiente Total (TGA) permite realçar as anomalias de curto comprimento de onda, fornecendo uma indicação do contorno dos corpos e da localização do seu centro. No entanto, observe que esta transformação exclui a informação referente ao caráter positivo ou negativo da anomalia, o qual por sua vez seria associado ao contraste de densidade dos corpos causadores destas anomalias. O mapa de TGA é calculado por:

$$ TGA = \sqrt{\left(\frac{\partial f}{\partial x}\right)^2 + \left(\frac{\partial f}{\partial y}\right)^2 + \left(\frac{\partial f}{\partial z}\right)^2 } $$

em que $f$ representa a amplitude da anomalia após a remoção do campo regional.

Execute a célula abaixo para visualizar o mapa da amplitude do gradiente total calculado sobre o mapa do campo residual.

In [ ]:
minha_aquisicao_tga = pg.gradiente_total(minha_aquisicao_res)
minha_aquisicao_tga.plota_mapa([-200.0, -200.0])

**Obs.:** Alguns autores referem-se à Amplitude do Gradiente Total pelo termo **Amplitude do Sinal Analítico (ASA)**, porém esta nomenclatura tem caído em desuso por ser uma nomenclatura menos adequada para representar o que de fato é calculado neste procedimento.

### Derivada *tilt* (T)

A derivada *tilt* é uma ferramenta de realce que permite estimar a geometria dos corpos a partir do cálculo do ângulo de inclinação (*tilt*) do gradiente, ou seja: 

$$T = tan^{-1}\left(\frac{{\partial f / \partial z}}{\sqrt{({\partial f / \partial x})^2 + ({\partial f / \partial y})^2}}\right)$$

A derivada *tilt* considera a normalização do gradiente vertical pelo gradiente horizontal total, permitindo visualizar no mapa tanto as anomalias causadas por fontes mais rasas quanto aquelas causadas por fontes mais profundas, atuando como uma espécie de ganho com controle automático (AGC) que equaliza as amplitudes das anomalias observas no mapa. Os picos das anomalias presentes no mapa resultante ficam posicionados sobre o centro dos corpos que as causam, porém este procedimento pode inserir artefatos no mapa, i.e., pode inserir falsas anomalias que não são de fato causadas por um contraste de densidade em profundidade. Desta forma, esta ferramenta deve ser utilizada com bastante atenção pelo intérprete.

Execute a célula abaixo para obter o mapa de *tilt derivative* a partir do mapa de campo residual. Observe como as anomalias agora apresentam aproximadamente a mesma amplitude entre si, com os picos localizados sobre as coordenadas centrais dos corpos. Observe também o surgimento de anomalias fortemente positivas nas coordenadas negativas de X e de uma anomalia negativa próximo ao corpo esférico.

In [ ]:
minha_aquisicao_tilt = pg.derivada_tilt(minha_aquisicao_res)
minha_aquisicao_tilt.plota_mapa([-200.0, -200.0])